In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/train.csv')

In [ ]:
# import os
# dir = 'D:\Covid-19\covid19-global-forecasting-week-1'
# os.chdir(dir)
# data = pd.read_csv('train.csv')

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.isnull().sum()/data.shape[0]*100

In [ ]:
data = data.drop('Province/State', axis=1)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
data.index = data.Date

In [ ]:
data = data.drop('Date', axis =1)
data = data.reset_index()
data.head()

In [ ]:
import plotly 
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
grp = data.groupby('Country/Region')['Country/Region', 'ConfirmedCases', 'Fatalities'].sum().reset_index()
fig = px.bar(grp, x = 'Country/Region', y = 'ConfirmedCases', title = 'Confirmed rate acroos country', log_y = True)
fig.show()

In [ ]:
fig = px.bar(grp, x = 'Country/Region', y = 'Fatalities', title = 'Fatalities rate acroos country', log_y = True)
fig.show()

In [ ]:
grp = data.groupby('Date')['Date', 'ConfirmedCases', 'Fatalities'].sum().reset_index()
fig = px.line(grp, x = 'Date', y = 'ConfirmedCases', title = 'Confirmed rate across country')
fig.show()

In [ ]:
grp = data.groupby('Date')['Date', 'ConfirmedCases', 'Fatalities'].sum().reset_index()
fig = px.line(grp, x = 'Date', y = 'ConfirmedCases', title = 'Fatalities rate across country')
fig.show()

In [ ]:
fig = go.Figure([go.Bar(x=pd.DataFrame(data[data['Country/Region']==i].groupby('Date')['ConfirmedCases'].sum()).index,
                        y=pd.DataFrame(data[data['Country/Region']==i].groupby('Date')['ConfirmedCases'].sum()).ConfirmedCases, name=str(i)) 
                 for i in data['Country/Region'].unique()],
               go.Layout(yaxis={'title': 'Total ConfirmedCases'},
                        title= ' ConfirmedCases for each Country/Region across days '))
fig.show()

In [ ]:
fig = go.Figure([go.Bar(x=pd.DataFrame(data[data['Country/Region']==i].groupby('Date')['Fatalities'].sum()).index,
                        y=pd.DataFrame(data[data['Country/Region']==i].groupby('Date')['Fatalities'].sum()).Fatalities, name=str(i)) 
                 for i in data['Country/Region'].unique()],
               go.Layout(yaxis={'title': 'Total Fatalities'},
                        title= ' Fatalities for each Country/Region across days '))
fig.show()

In [ ]:
data.head()

In [ ]:
import xgboost as xgb
data.index = data.Date
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Country/Region'] = le.fit_transform(data['Country/Region'])
data1 = data
data = data.drop(['Date','ConfirmedCases'], axis =1)
train = data[0:int(data.shape[0]*0.8)]
val = data[int(data.shape[0]*0.8): ]

In [ ]:
data1 = data1.drop(['Date','Fatalities'], axis =1)
train1 = data1[0:int(data1.shape[0]*0.8)]
val1 = data1[int(data1.shape[0]*0.8): ]

In [ ]:
data.dtypes

In [ ]:
print(data.shape)
print(train.shape)
print(val.shape)

In [ ]:
print(data1.shape)
print(train1.shape)
print(val1.shape)

In [ ]:
train_x = train.drop('Fatalities', axis=1)
val_x = val.drop('Fatalities', axis=1)
train_y = train[['Fatalities']]
val_y = val[['Fatalities']]

In [ ]:
train1.head()

In [ ]:
train1_x = train1.drop('ConfirmedCases', axis=1)
val1_x = val1.drop('ConfirmedCases', axis=1)
train1_y = train1[['ConfirmedCases']]
val1_y = val1[['ConfirmedCases']]

In [ ]:
model = xgb.XGBRegressor()
model1 = xgb.XGBRegressor()

In [ ]:
train1.head()

In [ ]:
model.fit(train_x, train_y, eval_set = [(val_x, val_y)])
model1.fit(train1_x, train1_y, eval_set = [(val1_x, val1_y)])

In [ ]:
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/test.csv')
test.Date = pd.to_datetime(test.Date)
test = test.drop(['Province/State', 'Date'], axis =1)
test['Country/Region'] = le.transform(test['Country/Region'])

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test.rename(columns = {"ForecastId": 'Id'}, inplace=True)

In [ ]:
test.head()

In [ ]:
a = model.predict(test)
b = model1.predict(test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
mean_absolute_error(model.predict(test), data[0:12212]['Fatalities'])

In [ ]:
mean_absolute_error(model1.predict(test), data1[0:12212]['ConfirmedCases'])

In [ ]:
test['Fatalities'] = a
test['ConfirmedCases'] = b

In [ ]:
test.rename(columns = {"Id": 'ForecastId'}, inplace=True)
test.head()

In [ ]:
test['Country/Region'] = le.inverse_transform(test['Country/Region'])

In [ ]:
test.head()

In [ ]:
test = test[['ForecastId', 'ConfirmedCases', 'Fatalities']]

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test = test[['ForecastId', 'ConfirmedCases', 'Fatalities']]

In [ ]:
test.to_csv('submission.csv',index= False)